In [3]:
from pyspark.sql import SparkSession
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Đường dẫn đến tệp CSV
SOURCE_CSV_FILE = os.path.abspath("./datasets/dataset-for-bank-loan-prediction.csv")

# Kiểm tra tệp có tồn tại không
if not os.path.isfile(SOURCE_CSV_FILE):
    raise FileNotFoundError(f"Tệp {SOURCE_CSV_FILE} không tồn tại.")

# Khởi tạo Spark Session
spark = SparkSession.builder \
    .appName("DataPipelineProject") \
    .getOrCreate()

# Đọc dữ liệu từ tệp CSV
try:
    df = spark.read.csv(SOURCE_CSV_FILE, header=True, inferSchema=True)
except Exception as e:
    print("Đã xảy ra lỗi khi đọc tệp CSV:", e)
    spark.stop()
    raise e

# Hiển thị vài dòng dữ liệu đầu tiên để kiểm tra
df.show()

# Lọc các hàng có giá trị không null trong các cột A, B và C
df_filtered = df.filter(df["Credit Score"].isNotNull() & df["Annual Income"].isNotNull())

# Nhóm dữ liệu theo cột A và tính tổng
df_grouped = df_filtered.groupBy("Credit Score").count()

# Hiển thị DataFrame bằng Pandas
pandas_df = df_grouped.toPandas()

# Hiển thị DataFrame đã được định dạng
styled_df = pandas_df.style.set_table_styles(
    [{'selector': 'tr:hover',
      'props': [('background-color', '#ffff99')]}]
).set_properties(**{
    'border': '1px solid black',
    'padding': '10px'
})

styled_df

# Vẽ biểu đồ thanh cho DataFrame đã nhóm
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.barplot(x='Credit Score', y='count', data=pandas_df)
plt.title("Số lượng theo A")
plt.show()

# Dừng Spark Session
spark.stop()

+--------------------+--------------------+-----------+-------------------+----------+------------+-------------+--------------------+--------------+------------------+------------+-----------------------+----------------------------+-----------------------+-------------------------+----------------------+-------------------+------------+---------+
|             Loan ID|         Customer ID|Loan Status|Current Loan Amount|      Term|Credit Score|Annual Income|Years in current job|Home Ownership|           Purpose|Monthly Debt|Years of Credit History|Months since last delinquent|Number of Open Accounts|Number of Credit Problems|Current Credit Balance|Maximum Open Credit|Bankruptcies|Tax Liens|
+--------------------+--------------------+-----------+-------------------+----------+------------+-------------+--------------------+--------------+------------------+------------+-----------------------+----------------------------+-----------------------+-------------------------+--------------

ValueError: Could not interpret value `A` for `x`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>